In [1]:
import numpy as np
import mlflow
import tensorflow as tf
print(tf.__version__)

ModuleNotFoundError: No module named 'tensorflow'

The Fashion MNIST data is available directly in the tf.keras datasets API

In [3]:
mnist = tf.keras.datasets.fashion_mnist

Images to train with the corresponding labels

In [4]:
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

4422102/4422102 [==============================] - 4s 1us/step


We notice that all of the values in the number are between 0 and 255. If we are training a neural network, for various reasons it's easier if we treat all values as between 0 and 1, a process called '**normalizing**'.

In [5]:
training_images  = training_images / 255.0
test_images = test_images / 255.0

Create a callback funditon which we will need later in the notebook. The function is used to cancel the training once an accuracy of 90% is reached.


In [11]:
class Callback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.9):
      print("\nReached 90% accuracy so cancelling training!")
      self.model.stop_training = True

Create a sequential model object:

In [6]:
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(), 
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

2023-01-16 10:36:11.793444: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


**Sequential**: That defines a **sequence** of layers in the neural network

**Flatten** Flatten takes that square and turns it into a 1 dimensional set/array.

**Dense**: Adds a layer of neurons. Each layer of neurons need an **activation function** to tell them what to do. 

**Relu** effectively means "If X>0 return X, else return 0" -- so what it does it it only passes values 0 or greater to the next layer in the network.

**Softmax** takes a set of values, and effectively picks the biggest one. If the output of the last layer looks like [0.2, 0.5, 0.01, 0.21, 6.5, 0.12, 0.2, 0.1, 0.34], it saves you from fishing through it looking for the biggest value, and turns it into [0,0,0,0,1,0,0,0,0]

Now we train the model...

In [13]:
callback = Callback()
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=3, callbacks = [callback])

Epoch 1/3
1875/1875 [==============================] - 12s 6ms/step - loss: 0.3118 - accuracy: 0.8853
Epoch 2/3
1875/1875 [==============================] - 13s 7ms/step - loss: 0.2932 - accuracy: 0.8921
Epoch 3/3
1875/1875 [==============================] - 9s 5ms/step - loss: 0.2804 - accuracy: 0.8954


...and evaluate it.

In [9]:
model.evaluate(test_images, test_labels)

313/313 [==============================] - 3s 8ms/step - loss: 0.3680 - accuracy: 0.8698


[0.36804887652397156, 0.8697999715805054]

In [10]:
classifications = model.predict(test_images)

print(classifications[0])
print(test_labels[0])

313/313 [==============================] - 3s 3ms/step
[4.5478468e-05 1.7836146e-06 2.8369041e-06 4.7360120e-07 2.6853699e-05
 6.1662220e-03 6.0962579e-05 2.6085349e-02 9.5100456e-04 9.6665901e-01]
9


The training accuracy is 88 %, which is not bad but also not great. For the test set the accuracy is 86 %, which says that our model is not overfitting the train data to well. Also consider that the model was trained only for 3 epochs. For a better resutl one can incresea the numner

# MLFLOW

The MLFLOW URI should not be stored on git. Save it locally in the .mlflow_uri file:

In [5]:
!echo http://127.0.0.1:5000/ > .mlflow_uri

In [7]:
!mlflow experiments create --experiment-name 0-ds-computer-vision

Created experiment '0-ds-computer-vision' with id 1


In [9]:
!mlflow ui

[2023-01-16 13:54:01 +0100] [10551] [INFO] Starting gunicorn 20.1.0
[2023-01-16 13:54:01 +0100] [10551] [INFO] Listening at: http://127.0.0.1:5000 (10551)
[2023-01-16 13:54:01 +0100] [10551] [INFO] Using worker: sync
[2023-01-16 13:54:01 +0100] [10553] [INFO] Booting worker with pid: 10553
^C
[2023-01-16 13:57:28 +0100] [10551] [INFO] Handling signal: int
[2023-01-16 13:57:28 +0100] [10553] [INFO] Worker exiting (pid: 10553)
